<a href="https://colab.research.google.com/github/dmbrundage88/genomic_datascience/blob/main/python_for_genomic_data_science_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount google drive
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
% pip install biopython

     |████████████████████████████████| 2.3MB 5.0MB/s 


In [3]:
import Bio
Bio.__version__

'1.78'

In [24]:
for record in SeqIO.parse("/content/gdrive/My Drive/dna2.fasta", "fasta"):
  for strand, nuc in [(+1, record.seq), (-1, record.seq.reverse_complement())]:
       for frame in range(3):
           length = 3 * ((len(record)-frame) // 3) #Multiple of three
           for pro in nuc[frame:frame+length].split("*"):
             if frame == 2:
                   print("%s...%s - length %i, strand %i, frame %i" % (pro[:30], pro[-3:], len(pro), strand, frame))

CGCGTTGCAGGCCGGCGTGTCGCGCAACGA...TGC - length 4632, strand 1, frame 2
CACCGCGCACCGCGTCGCCGAGCGGCTCGA...CGA - length 4632, strand -1, frame 2
CGATCCAGTAGCGCTTGTAGCCGAGCGCTT...GCC - length 1149, strand 1, frame 2
CCCAGGAGTCCTATACCAGCGTGACGACGT...GCG - length 1149, strand -1, frame 2
CGACGCGCTCCGCGTCGAGGTCGCCCGACG...GTT - length 4890, strand 1, frame 2
AACGCATAGACGTTGATGCCGTGATGCATG...TCG - length 4890, strand -1, frame 2
TGCTCGGCACGACTATCAGCCCGTATCTGT...CGC - length 3507, strand 1, frame 2
GCGAGCAGGCGGAGCGCATCTACGTGCAGC...GCA - length 3507, strand -1, frame 2
CATCGGCGACGGCTACTGGTTCCTCGAGAA...TCA - length 4074, strand 1, frame 2
AAAGCACGCAGGATCAGCAGGAACTCGATC...GCA - length 4074, strand -1, frame 2
CCGTGCCGACGCGCCGCATCGTCAGCGCGC...CGA - length 2865, strand 1, frame 2
GCGTGCGGAAGGCGAATGGGTGCCGAACGT...GGC - length 2865, strand -1, frame 2
CGGGCGGCCTGTGGGCGGTCGGCGCCGGGC...TTG - length 438, strand 1, frame 2
CAAGCAGCCGTCATTGCCGGGCCGCGACTG...CCG - length 438, strand -1, frame 2
ACCGGCGAGGAAGTG

In [4]:
# (1) How many records are in the file? A record in a FASTA file is defined as a single-line header, followed by lines of sequence data. 
# The header line is distinguished from the sequence data by a greater-than (">") symbol in the first column. The word following the ">" symbol is the identifier of the sequence, 
# and the rest of the line is an optional description of the entry. There should be no space between the ">" and the first letter of the identifier.

#read in fasta
from Bio import SeqIO
records = []
for record in SeqIO.parse("/content/gdrive/My Drive/dna2.fasta", "fasta"):
    records.append(record.id)
print(len(records))

18


In [7]:
# 2) What are the lengths of the sequences in the file? What is the longest sequence and what is the shortest sequence? 
# Is there more than one longest or shortest sequence? What are their identifiers
seq_dict = {}
for record in SeqIO.parse("/content/gdrive/My Drive/dna2.fasta", "fasta"):
    seq_dict[record.id]= len(record.seq)
lengths = sorted(list(seq_dict.values()))
min_length = min(lengths)
max_length = max(lengths)

for record in seq_dict:
  if seq_dict[record] == min_length:
    print('minimum length value: '+ record, min_length)
  if seq_dict[record] == max_length:
    print('maximum length value: '+ record, max_length)

maximum length value: gi|142022655|gb|EQ086233.1|255 4894
minimum length value: gi|142022655|gb|EQ086233.1|346 115


In [8]:
def find_orf(seq):

  stops = ["TAA","TGA","TAG"]    
  lst1 = [] #List for the start codons
  lst2 = [] #List for the stop codons
  start = 0 #The start position of the sequence.
  counter = 0 #Counter for 3 optional orfs.
  sequences =[]
  #initializes the lists for 3 optional orfs.
  for i in range (3):
      lst1.append([])
      lst2.append([])
  #Add to the lists the positions of the start and stop codons.
  while (seq and counter < 3):

      for i in range(start,len(seq),3):
          codon = seq[i:i+3] #The codon is 3 nucleotides.
          #print codon+ "\t"
          if(codon == "ATG"): #If the codon is  a start codon.
              lst1[start].append(i) #The position of the start codon.

          if(codon in stops): #if the codon is a stop codon.
              lst2[start].append(i+3) #The position of the stop codon.


      start += 1 #promotes the starting position.
      counter += 1 #promotes the counter
  for idx ,l in enumerate(lst1):
    for idx2, i in enumerate(l):
      try:
        sequences.append(seq[i:lst2[idx][idx2]])
      except(IndexError):
        sequences.append('error')
  output = [str(sequence) for sequence in sequences if sequence not in ['error','']]
  return output

In [9]:
# 3) In molecular biology, a reading frame is a way of dividing the DNA sequence of nucleotides into a set of consecutive, non-overlapping triplets (or codons). 
# Depending on where we start, there are six possible reading frames: three in the forward (5' to 3') direction and three in the reverse (3' to 5'). For instance, 
# the three possible forward reading frames for the sequence AGGTGACACCGCAAGCCTTATATTAGC are:

# AGG TGA CAC CGC AAG CCT TAT ATT AGC
# A GGT GAC ACC GCA AGC CTT ATA TTA GC
# AG GTG ACA CCG CAA GCC TTA TAT TAG C

# These are called reading frames 1, 2, and 3 respectively. An open reading frame (ORF) is the part of a reading frame that has the potential to encode a protein. 
# It starts with a start codon (ATG), and ends with a stop codon (TAA, TAG or TGA). For instance, ATGAAATAG is an ORF of length 9.

# Given an input reading frame on the forward strand (1, 2, or 3) your program should be able to identify all ORFs present in each sequence of the FASTA file, 
# and answer the following questions: 

# what is the length of the longest ORF in the file? 
# What is the identifier of the sequence containing the longest ORF? 
# For a given sequence identifier, what is the longest ORF contained in the sequence represented by that identifier? 
# What is the starting position of the longest ORF in the sequence that contains it? The position should indicate the character number in the sequence. 
# For instance, the following ORF in reading frame 1:

# >sequence1
# ATGCCCTAG
# starts at position 1.
# Note that because the following sequence:
# >sequence2
# ATGAAAAAA
# does not have any stop codon in reading frame 1, we do not consider it to be an ORF in reading frame 1.
orf_dict={}
for record in SeqIO.parse("/content/gdrive/My Drive/dna2.fasta", "fasta"):
  orf_dict[record.id] = find_orf(record.seq)


In [10]:
#longest ORF in file
orf_lengths=[]
for key in orf_dict:
  for value in orf_dict[key]:
    orf_lengths.append(len(value))
max_orf_length = max(orf_lengths)
print(max_orf_length)

3174


In [11]:
#identifier of sequence with longest orf
for key in orf_dict:
  for value in orf_dict[key]:
    if len(value) == max_orf_length:
      print(key)

gi|142022655|gb|EQ086233.1|16


In [12]:
# For a given sequence identifier, what is the longest ORF contained in the sequence represented by that identifier? 
def longest_orf(seq_id):
  seq_lengths = [len(seq) for seq in orf_dict[seq_id]]
  max_index = seq_lengths.index(max(seq_lengths))
  return max(seq_lengths), orf_dict[seq_id][max_index]
longest_orf('gi|142022655|gb|EQ086233.1|16')

(3174,
 'ATGCCGGCGTTGACGTTTTCGTGGAGTGAGTCAGATGAATCACGCAGCGAATCCCGCCGATCCCGATCGCGCCGCGGCGCAGGGCGGCAGCCTGTACAACGACGATCTCGCGCCGACGACGCCGGCGCAGCGCACGTGGAAGTGGTATCACTTCGCGGCGCTGTGGGTCGGGATGGTGATGAACATCGCGTCGTACATGCTCGCGGCCGGGCTGATCCAGGAAGGCATGTCGCCGTGGCAGGCGGTGACGACGGTGCTGCTCGGCAACCTGATCGTGCTCGTGCCGATGCTGCTGATCGGCCATGCGGGCGCGAAGCACGGGATTCCGTACGCGGTGCTCGTGCGCGCGTCGTTCGGCACGCAGGGGGCGAAGCTGCCGGCGCTGCTGCGCGCGATCGTCGCGTGCGGCTGGTACGGGATCCAGACCTGGCTCGGCGGCAGCGCGATCTATACGCTGCTGAACATCCTGACCGGCAACGCGCTGCATGGCGCCGCGCTGCCGGTCATCGGCATCGGGTTCGGGCAGCTCGCATGCTTCCTCGTGTTCTGGGCGCTGCAGCTCTACTTCATCTGGCATGGCACCGATTCGATCCGCTGGCTCGAAAGCTGGTCGGCGCCGATCAAGGTCGTGATGTGCGTGGCGCTGGTGTGGTGGGCAACGTCGAAGGCGGGCGGCTTCGGCACGATGCTGTCGGCGCCGTCGCAGTTTGCCGCAGGCGGCAAGAAAGCCGGGCTGTTCTGGGCGACCTTCTGGCCGGGGCTGACCGCGATGGTCGGCTTCTGGGCGACGCTCGCGCTGAACATCCCCGACTTCACGCGCTTCGCGCATTCGCAGCGCGACCAGGTGATCGGCCAGTCGATCGGGCTGCCGTTGCCGATGGCGCTGCTGTCGGTGGTGTCGGTCGTCGTGACGTCGGCGACCGTCGTGATCTACGGCAACGCGATCTGGGATCCGATCGACCTGACGAGCCGGATGACGGGCATCGGCGTG

In [16]:
#What is the starting position of the longest ORF in the sequence that contains it? The position should indicate the character number in the sequence.
def orf_start_id(seq_id):
  for record in SeqIO.parse("/content/gdrive/My Drive/dna2.fasta", "fasta"):
    if record.id == seq_id:
      sequence = record.seq
      length, substring = longest_orf(seq_id)
      substring_position = sequence.find(substring)+1
      return substring_position
orf_start_id('gi|142022655|gb|EQ086233.1|16')

1494

In [37]:
# '''(4) A repeat is a substring of a DNA sequence that occurs in multiple copies (more than one) somewhere in the sequence. 
# Although repeats can occur on both the forward and reverse strands of the DNA sequence, we will only consider repeats on the forward strand here. 
# Also we will allow repeats to overlap themselves. For example, the sequence ACACA contains two copies of the sequence ACA - once at position 1 (index 0 in Python), 
# and once at position 3. Given a length n, your program should be able to identify all repeats of length n in all sequences in the FASTA file. 
# Your program should also determine how many times each repeat occurs in the file, and which is the most frequent repeat of a given length.'''
def find_repeats(seq, n):
    dic_list=[]
    repeat_dic={}
    repeats = []
    for i in range(1,len(seq)):
        repeat = seq[i-1:i+n]
        repeats.append(repeat)
    repeat_set = set(repeats)
    for r in repeat_set:
      repeat_dic[str(r)] = repeats.count(r)
    dic_list.append(repeat_dic)
    return dic_list
total_repeats ={}
for record in SeqIO.parse("/content/gdrive/My Drive/dna2.fasta", "fasta"):
  seq_repeat = []
  seq_repeat.extend(find_repeats(record.seq,4))

In [38]:
import collections
counter = collections.Counter() 
for d in seq_repeat:  
    counter.update(d) 
      
result = dict(counter) 
  
  
print("resultant dictionary : ", str(counter))


resultant dictionary :  Counter({'CGCCG': 27, 'GCCGC': 27, 'CGCGA': 20, 'CGACG': 18, 'CCGCC': 16, 'CCGCG': 16, 'GGCCG': 15, 'CGGCC': 15, 'CATCG': 15, 'ACGCG': 15, 'CGCGG': 15, 'ATCGC': 14, 'CAGCC': 14, 'GCAGC': 13, 'GACGC': 13, 'GCGGC': 13, 'GCGAT': 13, 'CGGCG': 13, 'CCGGC': 12, 'CGACC': 12, 'GGCGA': 12, 'GCCGA': 12, 'TCGAC': 12, 'CGGCA': 12, 'GCCGG': 12, 'CAGGC': 11, 'TCGCC': 11, 'CGATG': 11, 'TCGCG': 11, 'CGCGC': 11, 'ACGCC': 11, 'CCAGC': 11, 'CAGCA': 11, 'ACGGC': 11, 'ACCGC': 11, 'TCGGC': 10, 'CGATC': 10, 'CCGAC': 10, 'GACGA': 10, 'GCTCG': 10, 'CGAAC': 10, 'AACGC': 10, 'GCGCG': 10, 'CCCGA': 9, 'GCGGA': 9, 'CCATC': 9, 'GACCA': 9, 'TGGCG': 9, 'GCCGT': 9, 'AGCCG': 9, 'AGGCC': 9, 'AGCAG': 9, 'GCGAC': 9, 'TGCCG': 9, 'GATCG': 9, 'CACCG': 9, 'CGTCG': 8, 'CCGGA': 8, 'CGCAG': 8, 'GATCA': 8, 'GTCGG': 8, 'GCACG': 8, 'CCGCA': 8, 'GGCCC': 8, 'GCCCG': 8, 'GCCAG': 8, 'CGCCC': 8, 'GCCTG': 8, 'CGCTG': 8, 'GCGAG': 8, 'ACCGG': 8, 'CAGCG': 7, 'GGTCG': 7, 'GACCG': 7, 'GCGTG': 7, 'CGGAA': 7, 'CTGGC': 7, 

In [ ]:
#Parse Blast result
from Bio.Blast import NCBIXML
blast_record = NCBIXML.read(result_handle)

E = 0.01
for alignment in blast_record.alignments:
  for hsp in alignment.hsps:
    if hsp.expect < E:
      print("****ALIGNMENT****")
      print("Sequence: ", alignment.title)
      print("Length: ", alignment.length)
      print("e value: ", hsp.expect)
      print(hsp.query)
      print(hsp.match)
      print(hsp.sbjct)


****ALIGNMENT****
Sequence:  gi|1783584753|gb|MN651324.1| Nicotiana tabacum strain zhongyan90 cytoplasmic male sterility(CMS) line cultivar MSzhongyan90 mitochondrion, complete genome
Length:  530869
e value:  5.81253e-96
TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTACAATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCACCTACGGTAGAG
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTACAATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCACCTACGGTAGAG
****ALIGNMENT****
Sequence:  gi|1783584659|gb|MN651323.1| Nicotiana tabacum strain zhongyan90 maintainer line cultivar zhongyan90 mitochondrion, complete genome
Length:  47221

In [ ]:
#get reverse compliment of sequence
from Bio.Seq import Seq
my_seq = Seq("TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTACAATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCACCTACGGTAGAG")
print('reverse complement is %s' % my_seq.reverse_complement())

reverse complement is CTCTACCGTAGGTGCCAAAGAAGAATGAGAAGAAGCCCGAAAAAGAGCAAAAAAGAACATAACCTCCGATACGATAAACAGAATAAAACCATATCGAGGTCCTAATTGTACGACTTTGGTATGATGTCCTTCGAACGTGGATTCACGTAGAACATCGCGCCACCATACGAACATGGTATATAGGATAAATATGAGGCCCA


In [ ]:
#get protein translation
my_seq = Seq("TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTACAATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCACCTACGGTAGAG")
my_seq.translate()


/usr/local/lib/python3.6/dist-packages/Bio/Seq.py:2338: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


Seq('WASYLSYIPCSYGGAMFYVNPRSKDIIPKSYN*DLDMVLFCLSYRRLCSFLLFF...LR*')